# Resolução de puzzles Sokoban

## Formulação

### Atributos

Foi escolhido representar um estado do puzzle sokoban usando uma class `EstadoSokoban` que tem como atributos:

- `tabuleiro`: lista de listas que representa o puzzle: Cada uma das litas interiores vão ter:

    - ’#’   – para representar as paredes;
    
    - ’.’   – para representar as posições livres;
    
    - ’*’   – para representar as caixas;
    
    - ’o’   – (um ó minúsculo) para representar os alvos;
    
    - ’A’   – para representar o arrumador.
    
    - ’@’   – para representar uma caixa numa posição alvo.
    
    - ’B’   – para representar o arrumador em cima de uma posição alvo.
    
- `arrumador`: posição do arrumador, tuplo xy.

- `caixas`: posição das caixas, tuplo xy.

- `alvos`: posição dos alvos, tuplo xy.

- `deadlocks`: posição de deadlocks, executando `deadlocks_tabuleiro()`

### Métodos

Para além da representação, foram implementados os seguintes métodos:

- `pos_livre(self, x, y)`: verifica se uma posição do puzzle está livre, isto é, se o arrumador por andar para lá.

- `pos_caixa(self, x, y)`: verifica se uma posição do puzzle tem uma caixa.

- `ver_*(self, x, y)`: usa as funções anteriores para verificar a posição "cima", "baixo", "direita" e "esquerda" (cada uma destas opções são um método diferente.

## Nota sobre este relatório

Utilizamos a ferramenta Jupyter Notebook, para tirar proveito de algumas funcionalidades do IPython para análise estatística da execução dos algoritmos.

## Heurísticas definidas

## Exemplos de execução

No ficheiro `sokoban.py` estão as classes principais para a execução do puzzle. Todo o código de análise execução (apresentado neste relatório) está no ficheiro `run-sokoban.py`.

In [1]:
%run -t sokoban.py


IPython CPU timings (estimated):
  User   :       0.01 s.
  System :       0.00 s.
Wall time:       0.01 s.


## Análise dos algoritmos experimentados

Vai ser utilizado o puzzle2, entregue no enunciado, para testar os algorimos disponíveis no ficheiro `search.py`, do repositório aima-python, disponibilizado nas aulas.

In [2]:
sokoban = Sokoban(puzzle2)

def statistics(resultado, verbose=False):
    '''Metodo concreto para imprimir dados da resolução de um problema Sokoban'''
    path = resultado.path()
    solucao = resultado.solution()
    number_moves = 0
    number_pushes = 0

    for index, action in enumerate(solucao):
        accao, _ = action.split()
        if accao == 'andar':
            number_moves += 1
        else:
            number_pushes += 1

    for index, state in enumerate(path):
        if verbose:
            print(state)
    else:
        print('Número de passos:', index)

    print('Números de moves:', number_moves)
    print('Números de pushes:', number_pushes)

In [3]:
%%timeit 
ucs_resultado = uniform_cost_search(sokoban)

1.62 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
bfs_resultado = breadth_first_search(sokoban)

1.47 s ± 53.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
bfs_resultado = breadth_first_search(sokoban)
statistics(bfs_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [6]:
ucs_resultado = uniform_cost_search(sokoban)
statistics(ucs_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [7]:
%%timeit
astar_resultado = astar_search(sokoban, heur_manhattan)

1.6 s ± 74.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
astar_resultado = astar_search(sokoban, heur_manhattan)
statistics(astar_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [42]:
def hung_alg_manh(nodo):
    """Algoritmo hungaro, em que o custo de cada caixa a um alvo é a distância de manhattan."""
    m = Munkres()
        
    caixas = nodo.state.caixas
    alvos = nodo.state.alvos
    custo = list()
    mhd = 0

    for index, c in enumerate(caixas):
        custo.append(list())
        for a in alvos:
            custo[index].append(abs(c[0] - a[0]) + abs(c[1] - a[1]))
    
    indexes = m.compute(custo)
    for row, column in indexes:
        value = custo[row][column]
        mhd += value
    
    print(mhd)
    return mhd

In [ ]:
sokoban1 = Sokoban(puzzle3)

astar_resultado = astar_search(sokoban1, hung_alg_manh)
statistics(astar_resultado)

12
12
12
12
12
12
12
12
12
12
13
12
12
12
12
12
12
13
12
13
13
12
13
12
12
12
12
13
13
13
13
13
13
12
12
12
13
13
14
13
13
12
13
13
13
13
13
12
12
12
12
12
12
13
13
14
14
13
13
13
12
13
12
14
13
13
13
13
13
12
13
12
12
12
12
12
14
14
14
14
14
13
13
13
13
12
13
13
13
14
13
14
13
13
13
12
13
12
13
12
12
12
12
13
13
13
14
14
14
14
14
13
13
13
13
13
12
13
14
13
14
13
13
14
13
15
14
12
13
13
12
12
13
14
12
12
14
13
12
13
14
13
12
13
13
13
11
14
15
14
14
14
13
11
11
14
14
14
13
13
14
14
14
15
14
12
13
14
13
12
12
13
14
14
12
13
13
12
13
14
13
14
14
14
14
12
13
14
14
13
12
14
14
14
14
13
14
13
12
12
11
11
14
13
11
11
14
14
14
13
11
15
14
16
14
14
14
14
13
15
14
14
13
14
13
13
14
14
15
14
15
14
13
13
12
13
12
13
13
13
13
14
14
13
13
14
15
13
14
13
13
13
12
14
15
13
13
15
14
13
14
15
13
14
14
13
13
12
13
13
15
14
13
14
15
13
14
14
14
13
13
13
13
13
15
14
13
14
12
12
12
11
11
14
13
13
11
11
14
13
11
15
16
15
14
15
14
14
14
13
13
13
13
13
14
13
14
15
15
15
14
15
15
14
14
13
13
12
12
13
13
12
13
1

15
14
14
14
14
14
15
14
14
15
15
14
14
14
14
14
13
13
13
13
13
13
14
14
13
13
13
14
14
14
13
13
13
14
14
14
14
14
14
14
15
14
14
15
16
15
15
15
15
15
14
15
14
14
15
15
15
15
15
15
14
14
14
15
15
15
15
15
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
11
11
11
12
12
11
11
14
14
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
13
13
15
15
14
14
15
15
15
14
15
15
14
14
15
15
15
15
15
14
15
15
15
15
15
14
14
14
14
14
16
16
17
16
16
15
16
16
15
15
16
16
16
16
15
15
16
13
12
11
10
13
12
11
10
11
11
13
13
11
11
13
13
12
12
11
11
15
15
13
13
13
14
14
12
12
12
12
12
10
10
10
9
9
13
13
13
12
12
10
10
10
14
14
15
15
15
13
13
14
14
14
12
12
12
12
10
10
10
14
14
14
13
13
13
11
11
11
11
13
13
13
13
13
13
13
11
11
11
11
13
13
12
12
10
10
10
11
15
15
14
15
14
13
15
15
14
14
13
13
14
14
14
13
14
13
12
14
14
14
14
13
13
12
12
12
11
13
12
12
12
11
11
12
11
11
11
12
11
12
13
13
13
13
13
13
13
13
14
13
12
13
12
13
13
13
13
13
11
11
11
11
11
10
10
10
10
11
11
11
12
11
11
10
11
10
10
10
10
10
10
10
10
9
9
9


14
14
13
14
14
13
12
12
12
11
11
11
11
10
10
10
9
9
9
14
14
14
15
15
15
15
14
14
15
14
14
14
14
16
16
16
13
11
12
10
10
9
9
8
7
12
11
10
8
13
11
11
11
11
10
10
10
9
9
9
8
8
14
14
13
13
13
13
12
12
11
11
11
9
9
9
13
13
13
13
14
14
14
13
13
13
13
13
12
12
10
10
9
9
7
7
12
12
10
10
13
13
13
13
13
12
12
12
11
11
11
10
10
10
9
9
9
7
7
7
12
12
12
10
10
10
10
10
10
8
8
8
13
14
14
13
13
12
13
13
12
12
12
13
13
10
10
10
9
9
9
8
8
8
14
14
13
13
13
13
12
12
12
12
13
13
13
12
12
12
11
11
11
10
10
10
9
9
9
7
7
7
11
11
10
10
8
11
11
11
11
11
11
11
10
9
9
9
9
8
8
8
9
9
9
8
8
8
8
8
8
8
7
7
7
7
8
7
7
8
8
8
7
8
7
7
15
14
13
13
14
13
12
12
12
12
11
11
11
13
12
12
12
12
11
11
11
11
11
11
11
11
9
9
9
13
13
12
12
12
12
10
10
10
11
12
12
12
12
11
10
9
9
9
9
9
8
8
8
8
8
9
9
12
13
12
13
13
13
13
13
12
13
13
12
12
13
13
13
12
12
12
12
13
13
13
13
13
14
14
14
13
14
14
14
14
13
13
13
13
13
13
14
14
13
13
14
14
13
13
13
13
13
13
15
14
13
9
11
14
14
14
14
14
14
13
13
14
14
15
15
15
15
14
14
11
11
12
12
13
13
14
14


15
15
15
15
14
11
12
11
11
11
12
14
13
13
14
13
14
13
13
11
10
17
17
16
16
16
16
15
15
14
14
13
14
13
13
11
12
12
11
11
11
12
12
12
11
11
12
11
10
10
10
10
10
11
11
11
12
12
11
11
11
11
11
11
17
17
16
16
16
16
15
15
15
16
14
14
15
18
18
17
16
16
16
16
15
15
17
18
17
17
17
16
16
16
16
16
16
16
16
16
16
16
16
16
15
16
15
16
16
16
16
16
16
15
15
15
15
15
15
15
15
15
15
15
14
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
13
12
13
13
11
12
11
11
10
11
10
10
10
12
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
13
11
11
14
14
16
16
16
15
15
15
15
16
15
14
14
15
16
17
17
16
17
17
16
17
17
15
15
15
16
16
16
16
15
14
13
15
15
15
15
14
14
14
14
14
14
14
14
15
15
15
15
15
14
14
14
14
15
15
15
15
15
13
12
15
14
13
13
14
14
14
13
13
14
14
13
13
14
13
14
13
13
14
12
13
12
13
13
13
12
12
13
13
14
14
14
13
14
13
14
14
14
14
14
14
14
13
13
13
13
13
13
14
14
14
14
14
14
14
14
17
15
10
11
16
16
16
16
15
15
15
15
15
13
17
17
17
16
17
16
17
17
17
16
16
16
16
15
15
15
15
15
15
15
15
15
16
15
15
15
15
15
15
15
1

12
13
13
13
13
13
14
14
14
13
13
13
14
14
14
14
13
13
13
14
14
14
14
13
14
14
11
11
11
9
10
10
11
11
11
12
13
13
13
12
11
10
10
10
9
9
9
9
9
8
8
8
8
13
13
13
14
14
14
13
15
15
14
15
15
14
14
15
11
12
12
15
15
14
14
15
15
15
15
14
14
14
14
13
15
15
11
11
15
15
10
16
16
15
14
13
11
11
12
12
11
12
12
14
14
14
14
14
14
13
15
15
15
15
15
13
13
11
11
13
13
13
13
13
13
13
11
11
14
14
14
14
14
13
13
12
12
12
12
11
11
11
11
11
11
11
11
11
11
12
12
12
11
11
11
11
12
13
11
11
12
12
12
12
12
11
11
11
10
10
11
10
10
10
10
10
11
11
11
11
11
11
11
13
13
12
12
12
12
11
11
11
11
11
11
11
11
12
11
11
13
14
14
18
18
18
18
17
17
17
17
16
16
16
16
16
16
16
15
18
18
18
17
17
17
17
17
17
16
16
16
16
16
16
16
16
16
16
16
16
16
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
14
15
15
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
12
12
12
10
11
11
11
11
10
9
10
10
10
10
10
12
12
14
13
14
14
15
15
15
15
15
15
15
14


14
13
13
11
8
15
12
12
12
11
11
11
10
11
8
8
7
12
11
10
9
9
9
9
12
13
13
12
12
11
9
9
12
12
11
11
9
9
11
11
11
10
12
12
10
10
12
13
14
12
12
10
10
12
12
12
11
11
11
9
9
9
11
11
11
11
11
11
10
12
12
11
11
12
12
12
11
10
10
10
9
9
13
13
12
12
15
15
14
14
13
14
14
13
13
13
11
11
11
12
12
10
10
11
11
10
10
9
9
8
8
7
7
12
12
12
11
11
10
10
8
8
8
12
13
13
13
11
11
12
12
10
10
11
11
11
10
10
9
9
8
8
7
7
12
12
12
11
11
11
9
9
9
9
11
11
11
11
11
11
11
10
9
9
9
8
8
12
13
11
11
12
10
11
10
10
9
9
8
8
7
7
8
9
13
12
13
12
11
13
13
12
12
11
11
12
12
12
11
12
11
10
12
12
12
12
11
11
10
10
10
9
11
10
10
10
9
9
9
10
9
9
9
9
10
10
10
9
9
10
11
11
11
11
11
11
12
11
10
11
10
10
10
11
11
11
11
11
11
9
9
9
9
9
8
9
9
9
9
8
8
8
8
8
8
8
8
9
10
9
9
8
8
8
8
8
7
7
8
8
8
8
8
8
7
7
7
7
7
7
7
8
8
12
12
12
13
13
14
13
14
14
13
14
14
13
13
13
13
13
13
13
13
12
13
13
13
13
12
12
12
12
12
12
12
12
12
12
12
12
11
12
12
12
12
12
12
12
12
13
12
12
11
11
11
11
11
12
12
11
11
10
10
9
9
9
8
8
8
9
9
9
12
12
12
11
11
11
10
10
1

11
11
10
10
13
13
12
12
17
17
12
12
12
12
13
13
13
13
12
11
10
13
13
16
14
15
14
14
13
12
12
11
9
9
16
14
14
14
13
13
13
13
13
13
12
12
12
11
11
11
10
10
10
13
13
13
12
12
12
12
12
12
17
17
17
14
12
12
12
13
12
12
13
13
13
13
13
13
12
12
12
11
11
11
10
10
13
13
16
15
14
14
13
13
13
12
12
12
12
12
9
9
9
13
11
10
10
10
10
10
14
14
13
14
14
13
13
12
12
12
12
12
12
11
9
9
14
14
14
13
13
13
13
13
13
12
12
12
11
11
11
11
10
10
10
13
13
13
13
12
12
12
12
12
12
10
10
9
14
13
13
12
12
12
11
9
10
10
11
13
13
14
13
13
13
14
12
14
13
14
14
14
14
13
13
12
13
12
12
11
12
12
13
13
12
13
13
13
13
12
12
11
12
11
11
11
10
10
11
11
11
11
10
10
11
11
10
10
10
10
10
10
12
12
12
12
11
13
13
13
13
12
12
12
12
13
12
12
12
12
12
11
12
11
12
12
12
12
12
12
12
12
12
12
11
11
11
10
10
10
10
11
10
10
10
9
8
10
10
10
10
10
10
10
9
9
9
9
9
10
11
10
10
9
10
9
9
8
8
9
9
9
9
9
9
9
9
9
9
9
9
8
8
8
8
8
8
14
14
14
14
13
14
15
15
15
15
16
15
15
14
15
15
15
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
13
13
1